Full Name: MohammadDavood VahhabRajaee

Student ID: 4041419041

# Pandas & Cleaning a Messy DataFrame

**Goal:** learn how to load a CSV into a `pandas.DataFrame`, inspect it, and clean common problems step-by-step:
- missing values, inconsistent column names, wrong data types, duplicates, stray whitespace and malformed values.
- By the end you'll practice the same steps on a second messy dataset.

## 1.Import libraries

In [1]:
import pandas as pd        # core library for tabular data
import numpy as np         # numerical helpers
import matplotlib.pyplot as plt
%matplotlib inline

## 2.Load the messy CSV

We will load the messy CSV directly from the GitHub repository (raw URL).  
**Note for students:** if the raw URL doesn't work for you (GitHub limits or network restrictions), download the file locally and upload it into Colab, or clone the repo.

Replace `RAW_URL` with the raw file path if necessary.


In [2]:
# load dataset (example raw URL from the 'Messy-dataset' GitHub repo)
RAW_URL = "https://raw.githubusercontent.com/eyowhite/Messy-dataset/main/messy_IMDB_dataset.csv"

# read_csv: automatically guesses separators and dtypes; returns a DataFrame
df = pd.read_csv(RAW_URL, low_memory=False, encoding='latin-1', delimiter=';')  # low_memory=False reduces dtype guess issues
df.head()

,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


### Explanation: `pd.read_csv()`
- `pd.read_csv(url)` reads a CSV and returns a **DataFrame** — pandas’ 2D table object (rows & columns).  
- `low_memory=False` helps pandas inspect the file more robustly before choosing data types (useful for messy files).
- `.head()` displays the first 5 rows so we can *see* how the file looks.

## 3.Quick inspection — understand what “messy” means

Use these basic commands to get a sense of the table:
- `df.shape` → rows × columns
- `df.columns` → column names
- `df.info()` → dtypes and non-null counts
- `df.describe(include='all')` → summary stats (for numeric and non-numeric)

In [3]:
# Inspection
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nInfo:")
df.info()
print("\nDescribe (all cols):")
df.describe(include='all').transpose()

Shape: (101, 12)

Columns: ['IMBD title ID', 'Original titlÊ', 'Release year', 'Genrë¨', 'Duration', 'Country', 'Content Rating', 'Director', 'Unnamed: 8', 'Income', ' Votes ', 'Score']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMBD title ID   100 non-null    object 
 1   Original titlÊ  100 non-null    object 
 2   Release year    100 non-null    object 
 3   Genrë¨          100 non-null    object 
 4   Duration        99 non-null     object 
 5   Country         100 non-null    object 
 6   Content Rating  77 non-null     object 
 7   Director        100 non-null    object 
 8   Unnamed: 8      0 non-null      float64
 9   Income          100 non-null    object 
 10   Votes          100 non-null    object 
 11  Score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB

Describe (all cols

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
IMBD title ID,100,100,tt0045152,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Original titlÊ,100,100,Singin' in the Rain,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Release year,100,99,2000-05-19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genrë¨,100,59,Drama,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Duration,99,71,149,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,100,18,USA,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Content Rating,77,7,R,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Director,100,64,Christopher Nolan,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Income,100,100,$ 1864182,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Explanation:
- `.shape` tells how many rows and columns we have.
- `.columns` shows column names (messy files often have bad names, extra whitespace, or duplicated names).
- `.info()` reveals non-null counts — a quick way to spot missing values.
- `.describe(include='all')` gives counts, unique values, top items for object columns and numeric stats for numbers.


## 4.Look for obvious problems (head, tail, random sample)

- Use `df.head()` and `df.tail()` to inspect top and bottom.
- Use `df.sample(10)` to inspect random rows — messy rows sometimes hide near the bottom or in random places.

In [4]:
df.head(8)

,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"
5,tt0167260,The Lord of the Rings: The Return of the King,22 Feb 04,"Action, Adventure, Drama",201,New Zealand,PG-13,Peter Jackson,NaN,$ 1142271098,1.604.280,08.9
6,tt0108052,Schindler's List,1994-03-11,"Biography, Drama, History",Nan,USA,R,Steven Spielberg,NaN,$ 322287794,1.183.248,8.9
7,tt0050083,12 Angry Men,1957-09-04,"Crime, Drama",96,USA,Not Rated,Sidney Lumet,NaN,$ 576,668.473,8.9


In [5]:
df.tail(6)

,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
95,tt0056172,Lawrence of Arabia,1963-10-29,"Adventure, Biography, Drama",228,UK,PG,David Lean,NaN,$ 45710874,261.504,7.5
96,tt0070735,The Sting,1974-03-21,"Comedy, Crime, Drama",129,USA,PG,George Roy Hill,NaN,$ 156000000,236.285,7.5
97,tt0082096,Das Boot,1982-03-18,"Adventure, Drama, Thriller",149,West Germany,R,Wolfgang Petersen,NaN,$ 11487676,226.427,7.5
98,tt0059578,Per qualche dollaro in piÃ¹,1965-12-20,Western,132,Italy,NaN,Sergio Leone,NaN,$ 15000000,226.039,7.4
99,tt1832382,Jodaeiye Nader az Simin,2011-10-21,Drama,123,Iran,PG-13,Asghar Farhadi,NaN,$ 22926076,214.165,7.4
100,tt0045152,Singin' in the Rain,1953-02-05,"Comedy, Musical, Romance",103,USA,NaN,Stanley Donen,NaN,$ 1864182,213.152,7.4


In [6]:
df.sample(8, random_state=42)

,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
84,tt0075314,Taxi Driver,1976-13-24,"Crime, Drama",114,USA,R,Martin Scorsese,NaN,$ 28441292,703.264,7.7
55,tt0082971,Raiders of the Lost Ark,1981-06-12,"Action, Adventure",115,USA,PG,Steven Spielberg,NaN,$ 390133212,865.510,8.2
66,tt1187043,3 Idiots,2009-12-25,"Comedy, Drama",170,India,NaN,Rajkumar Hirani,NaN,$ 60262836,332.217,8.1
67,tt0119698,Mononoke-hime,2000-05-19,"Animation, Adventure, Fantasy",134,Japan,PG-13,Hayao Miyazaki,NaN,$ 169785629,331.045,8.0
45,tt0034583,Casablanca,21-11-46,"Drama, Romance, War",102,USA,PG,Michael Curtiz,NaN,$ 4374761,509.953,8.3
39,tt0110357,The Lion King,1994-11-25,"Animation, Adventure, Drama",88,USA,G,"Roger Allers, Rob Minkoff",NaN,$ 968511805,917.248,8.4
22,tt0076759,Star Wars,1977-10-20,"Action, Adventure, Fantasy",121,USA,PG,George Lucas,NaN,$ 775768912,1.204.107,8.6
44,tt0054215,Psycho,1960-10-28,"Horror, Mystery, Thriller",109,USA,R,Alfred Hitchcock,NaN,$ 32008644,586.765,8.3


## 5.Clean column names

Common issues:
- Leading/trailing whitespace in column names
- Mixed case or separator characters (spaces, dots)
- Duplicated column names

We’ll:
1. Strip whitespace
2. Lowercase names (optional but helpful)
3. Replace spaces with underscores

Functions used:
- `df.columns` returns an index of names
- `str.strip()`, `str.lower()`, `str.replace()` operate on the index

In [7]:
# Clean column names
orig_cols = df.columns.tolist()
clean_cols = [col.strip().lower().replace(" ", "_").replace("-", "_") for col in orig_cols]
df.columns = clean_cols
print("Old columns:", orig_cols[:8])
print("New columns:", df.columns.tolist()[:8])

Old columns: ['IMBD title ID', 'Original titlÊ', 'Release year', 'Genrë¨', 'Duration', 'Country', 'Content Rating', 'Director']
New columns: ['imbd_title_id', 'original_titlê', 'release_year', 'genrë¨', 'duration', 'country', 'content_rating', 'director']


### Explanation:
We created a new list `clean_cols` and reassigned it to `df.columns`.  
This is safe and makes later code easier (`df.some_column` or `df["some_column"]`).

## 6.Identify missing values and their pattern

Use:
- `df.isna().sum()` to count missing values per column
- `df[df.isna().any(axis=1)].head()` to peek rows with any missing values

In [8]:
# Count missing values per column
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

unnamed:_8        101
content_rating     24
duration            2
imbd_title_id       1
genrë¨              1
release_year        1
original_titlê      1
country             1
director            1
income              1
votes               1
score               1
dtype: int64

In [9]:
# Peek rows that contain any missing values
df[df.isna().any(axis=1)].head(8)

,imbd_title_id,original_titlê,release_year,genrë¨,duration,country,content_rating,director,unnamed:_8,income,votes,score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"
5,tt0167260,The Lord of the Rings: The Return of the King,22 Feb 04,"Action, Adventure, Drama",201,New Zealand,PG-13,Peter Jackson,NaN,$ 1142271098,1.604.280,08.9
6,tt0108052,Schindler's List,1994-03-11,"Biography, Drama, History",Nan,USA,R,Steven Spielberg,NaN,$ 322287794,1.183.248,8.9
7,tt0050083,12 Angry Men,1957-09-04,"Crime, Drama",96,USA,Not Rated,Sidney Lumet,NaN,$ 576,668.473,8.9


### Explanation:
- `.isna()` returns a boolean DataFrame showing where values are NA/NaN.
- `.sum()` on booleans counts `True` values (i.e., missing entries).
- Spotting columns with many missing values helps decide whether to drop or impute them.

## 7.Duplicates

Check for duplicate rows (entire row identical) or duplicates on a key column (e.g., `title`, `id`).

Functions:
- `df.duplicated()` returns boolean Series (default: mark duplicates except first)
- `df.drop_duplicates()` removes duplicates

In [10]:
# Count duplicate full rows
num_dup = df.duplicated().sum()
print("Full-row duplicates:", num_dup)

# If duplicates exist, inspect a few
if num_dup:
    display(df[df.duplicated(keep=False)].head(8))

# Example: drop full-row duplicates (if appropriate)
df = df.drop_duplicates().reset_index(drop=True)
print("New shape after dropping exact duplicates:", df.shape)

Full-row duplicates: 0
New shape after dropping exact duplicates: (101, 12)


## 8.Fix data types (convert columns to appropriate types)

Common conversions:
- Strings representing numbers → `pd.to_numeric(..., errors='coerce')`
- Dates → `pd.to_datetime(..., errors='coerce')`

We use `errors='coerce'` so invalid parsing becomes `NaT`/`NaN`, which we can then inspect/impute.

In [11]:
# Example: try converting columns named 'year', 'duration', or 'votes' if they exist
for col in ["year", "duration", "votes", "rating"]:
    if col in df.columns:
        print(f"Converting {col} to numeric (if present)...")
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Example: parse a date-like column
for col in df.columns:
    if "date" in col or "released" in col:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Show dtypes after conversion
df.dtypes

Converting duration to numeric (if present)...
Converting votes to numeric (if present)...


imbd_title_id      object
original_titlê     object
release_year       object
genrë¨             object
duration          float64
country            object
content_rating     object
director           object
unnamed:_8        float64
income             object
votes             float64
score              object
dtype: object

### Explanation:
- `pd.to_numeric(series, errors='coerce')` turns strings into numbers when possible; invalid entries become `NaN`.
- `pd.to_datetime(..., errors='coerce')` similarly parses dates; bad formats → `NaT`.
- After conversion, re-run `df.info()` to confirm non-null counts and types.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imbd_title_id   100 non-null    object 
 1   original_titlê  100 non-null    object 
 2   release_year    100 non-null    object 
 3   genrë¨          100 non-null    object 
 4   duration        94 non-null     float64
 5   country         100 non-null    object 
 6   content_rating  77 non-null     object 
 7   director        100 non-null    object 
 8   unnamed:_8      0 non-null      float64
 9   income          100 non-null    object 
 10  votes           69 non-null     float64
 11  score           100 non-null    object 
dtypes: float64(3), object(9)
memory usage: 9.6+ KB


## 9.Handle inconsistent string formatting (strip, lower, replace)

String columns often have:
- Extra spaces
- Mixed capitalization
- Different separators in the same column (e.g., "Drama|Comedy" vs "Drama, Comedy")

We apply:
- `.str.strip()` to remove whitespace
- `.str.lower()` if we want normalized categories
- `.str.replace()` to standardize separators

In [13]:
# Example: clean string columns safely
for col in df.select_dtypes(include=["object"]).columns:
    # only do light cleaning if column exists and is non-empty
    df[col] = df[col].astype(str).str.strip()          # remove surrounding whitespace
    # Replace empty strings that may have been 'nan' after conversion
    df[col] = df[col].replace({"nan": np.nan, "None": np.nan, "": np.nan})

# Example: unify genre separators if a 'genre' column exists
if "genre" in df.columns:
    df["genre"] = df["genre"].str.replace("|", ",").str.replace(";", ",")

## 10.Impute or drop missing values (strategy)

Choose strategy based on column and task:
- Drop columns with almost all missing values: `df.drop(columns=[...])`
- Drop rows with too many missing fields: `df.dropna(thresh=some_number)`
- Fill numeric NaNs with median or mean: `df[col].fillna(df[col].median(), inplace=True)`
- Fill categorical NaNs with a placeholder: `df[col].fillna("unknown", inplace=True)`

Always **explain your choice** when cleaning.

In [14]:
# Example strategies: inspect % missing per column, then impute/drop
pct_missing = (df.isna().mean() * 100).sort_values(ascending=False)
pct_missing.head(12)

unnamed:_8        100.000000
votes              31.683168
content_rating     23.762376
duration            6.930693
genrë¨              0.990099
release_year        0.990099
original_titlê      0.990099
imbd_title_id       0.990099
director            0.990099
country             0.990099
income              0.990099
score               0.990099
dtype: float64

In [15]:
# Drop columns with > 80% missing
cols_to_drop = pct_missing[pct_missing > 80].index.tolist()
print("Dropping columns with >80% missing:", cols_to_drop)
df = df.drop(columns=cols_to_drop)

# For numeric columns, fill missing with median (example)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    median = df[col].median()
    df[col] = df[col].fillna(median)

# For categorical/object columns, fill with 'unknown'
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
for col in obj_cols:
    df[col] = df[col].fillna("unknown")

Dropping columns with >80% missing: ['unnamed:_8']


### Explanation:
- Dropping high-missing columns can be OK if they add no value.  
- Imputing numeric columns with the `median` is robust to outliers.  
- Categorical missing values can be filled with a specific token so models or later code can handle them.

## 11.Create derived columns / fix inconsistent units

Examples:
- Split a `full_name` into `first_name` and `last_name`
- Extract year from a date: `df['year'] = df['release_date'].dt.year`
- Convert currency string like `"$1,234.56"` to numeric by removing `$` and `,`.

Demonstration: clean a `box_office` column if it contains currency strings.


In [16]:
# Example: parse numeric values from a currency-like column
if "box_office" in df.columns:
    df["box_office_clean"] = (df["box_office"]
                              .astype(str)
                              .str.replace(r"[^0-9.]", "", regex=True)   # remove $ and commas
                              .replace("", np.nan)
                             )
    df["box_office_clean"] = pd.to_numeric(df["box_office_clean"], errors="coerce").fillna(0)
    df.drop(columns=["box_office"], inplace=True)
    df = df.rename(columns={"box_office_clean": "box_office"})
    print("Converted box_office to numeric.")

## 12.Validate and final checks

- Re-check `df.info()` and `df.isna().sum()` to confirm cleaning effect.
- Look at value counts for categorical columns: `df['genre'].value_counts().head()`
- Quick plots or histograms to detect remaining anomalies.

In [17]:
print(df.info())
print("\nMissing after cleaning:")
print(df.isna().sum().sort_values(ascending=False).head(10))

# Example: show distribution of a numeric column
if "rating" in df.columns:
    df["rating"].hist(bins=20)
    plt.title("Rating distribution")
    plt.xlabel("rating")
    plt.show()

# Example value counts
for col in ["genre", "country", "content_rating"]:
    if col in df.columns:
        print(f"\nTop values for {col}:")
        print(df[col].value_counts().head(8))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imbd_title_id   101 non-null    object 
 1   original_titlê  101 non-null    object 
 2   release_year    101 non-null    object 
 3   genrë¨          101 non-null    object 
 4   duration        101 non-null    float64
 5   country         101 non-null    object 
 6   content_rating  101 non-null    object 
 7   director        101 non-null    object 
 8   income          101 non-null    object 
 9   votes           101 non-null    float64
 10  score           101 non-null    object 
dtypes: float64(2), object(9)
memory usage: 8.8+ KB
None

Missing after cleaning:
imbd_title_id     0
original_titlê    0
release_year      0
genrë¨            0
duration          0
country           0
content_rating    0
director          0
income            0
votes             0
dtype: int64

Top values for

## 13.Save cleaned data

Save to CSV for later use:
- `df.to_csv("cleaned_imdb.csv", index=False)` saves locally in Colab VM.
- To keep it permanently, mount Google Drive in Colab and save there.

In [18]:
# Save cleaned file locally in Colab environment
df.to_csv("cleaned_imdb.csv", index=False)
print("Saved cleaned_imdb.csv (in Colab VM).")

Saved cleaned_imdb.csv (in Colab VM).


## 14.Recap — suggested cleaning checklist (short)
1. Inspect shape, columns, and dtypes: `df.shape`, `df.columns`, `df.info()`  
2. View head/tail and random samples: `df.head()`, `df.tail()`, `df.sample()`  
3. Clean column names (strip/lower/replace)  
4. Count missing values: `df.isna().sum()`  
5. Drop full-row duplicates: `df.drop_duplicates()`  
6. Convert dtypes: `pd.to_numeric`, `pd.to_datetime`  
7. Trim/normalize strings: `.str.strip()`, `.str.lower()`  
8. Impute or drop missing values appropriately  
9. Validate with `df.info()`, `value_counts()`, and plots  
10. Save cleaned dataset



# Practice — Clean a 2nd Messy Dataset

**Task:** practice the same pipeline on the *Cafe Sales - Dirty Data* dataset (Kaggle). Steps to follow:

1. Inspect with `df.head()`, `df.info()` and `df.isna().sum()`.  
2. Clean column names (strip/lower/underscore).  
3. Fix date/time columns (`pd.to_datetime`) and parse currency numbers.  
4. Handle missing values (drop or impute), remove duplicates.  
5. Produce a small report: total sales per day (groupby + sum) and top 5 products by revenue.  
6. Save cleaned dataset as `cleaned_cafe_sales.csv`.

**Hints & functions to use:**
- `pd.read_csv()`, `df.columns`, `df.isna().sum()`, `df.duplicated()`, `df.drop_duplicates()`, `pd.to_datetime()`, `pd.to_numeric(..., errors='coerce')`, `df.groupby(['date']).agg({'sales':'sum'})`, `df.to_csv()`.



In [19]:
import pandas as pd

### Step 0: Load the dataset

In [20]:
df = pd.read_csv("dirty_cafe_sales.csv")

### Step 1: Inspect the dataset

In [21]:
print("First 5 rows:")
print(df.head())
print("\nInfo about the dataset:")
print(df.info())
print("\nMissing values per column:")
print(df.isna().sum())

First 5 rows:
  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  

Info about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2 

### Step 2: Clean column names

In [22]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("\nCleaned column names:")
print(df.columns)


Cleaned column names:
Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')


### Step 3: Handle 'UNKNOWN' and fix date/number columns

In [23]:
df.replace('UNKNOWN', pd.NA, inplace=True)  # convert 'UNKNOWN' to NaN
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')  # convert to datetime

#### Convert numeric columns

In [24]:
numeric_cols = ['quantity', 'price_per_unit', 'total_spent']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

### Step 4: Remove duplicates and handle missing values

In [25]:
df = df.drop_duplicates()
df = df.dropna(subset=['transaction_date','total_spent','item','quantity','price_per_unit'])

### Step 5: Produce a small report
#### Total sales per day

In [26]:
sales_per_day = df.groupby('transaction_date')['total_spent'].sum().reset_index()
print("\nTotal sales per day:")
print(sales_per_day)


Total sales per day:
    transaction_date  total_spent
0         2023-01-01        127.5
1         2023-01-02        114.5
2         2023-01-03        128.0
3         2023-01-04        223.5
4         2023-01-05        288.0
..               ...          ...
360       2023-12-27        163.0
361       2023-12-28        169.0
362       2023-12-29         93.5
363       2023-12-30        132.0
364       2023-12-31        106.0

[365 rows x 2 columns]


#### Top 5 products by revenue

In [27]:
top_products = df.groupby('item')['total_spent'].sum().sort_values(ascending=False).head(5)
print("\nTop 5 products by revenue:")
print(top_products)


Top 5 products by revenue:
item
Salad       14195.0
Sandwich    10992.0
Smoothie    10888.0
Juice        8631.0
Cake         8577.0
Name: total_spent, dtype: float64


### Step 6: Save cleaned dataset

In [28]:
df.to_csv("cleaned_cafe_sales.csv", index=False)
print("\nCleaned dataset saved as 'cleaned_cafe_sales.csv'")


Cleaned dataset saved as 'cleaned_cafe_sales.csv'
